In [1]:
#importing required libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import numpy as np

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pandas as pd
from pandas.tseries.offsets import BDay

# pd.datetime is an alias for datetime.datetime
today = pd.datetime.today()
lbd = (today - BDay(1)).date()
url = 'https://www.quandl.com/api/v3/databases/EOD/metadata?#api_key=5czsVZzyn5zaXksb31tF'
tickers = pd.read_csv(url,engine='python', compression='zip')
df = tickers.loc[tickers['to_date'] == str(lbd)].reset_index(drop=True)
df.head()

,code,name,description,refreshed_at,from_date,to_date
0,A,"Agilent Technologies Inc. (A) Stock Prices, Di...",<p><b>Ticker</b>: A</p> <p><b>Exchange</b>: NY...,2019-05-07 00:48:12,1999-11-18,2019-05-06
1,AA,"Alcoa Corporation (AA) Stock Prices, Dividends...",<p><b>Ticker</b>: AA</p> <p><b>Exchange</b>: N...,2019-05-07 00:48:32,2016-11-01,2019-05-06
2,AAAU,"Perth Mint Physical Gold (AAAU) Stock Prices, ...",<p><b>Ticker</b>: AAAU</p> <p><b>Exchange</b>:...,2019-05-07 00:48:53,2018-08-15,2019-05-06
3,AABA,"Altaba Inc. (AABA) Stock Prices, Dividends and...",<p><b>Ticker</b>: AABA</p> <p><b>Exchange</b>:...,2019-05-07 00:48:12,1996-04-12,2019-05-06
4,AAC,"AAC Holdings Inc. (AAC) Stock Prices, Dividend...",<p><b>Ticker</b>: AAC</p> <p><b>Exchange</b>: ...,2019-05-07 00:48:24,2014-10-02,2019-05-06


# Get monthly

In [6]:
from alpha_vantage.timeseries import TimeSeries
key = 'BJUEJ3R9ZM8ER5KY'
ts = TimeSeries(key=key)

In [7]:
data, meta_data = ts.get_monthly(symbol='A')
data

{'1999-12-31': {'1. open': '42.1900',
  '2. high': '80.0000',
  '3. low': '41.0000',
  '4. close': '77.3100',
  '5. volume': '47860100'},
 '2000-01-31': {'1. open': '78.7500',
  '2. high': '78.9400',
  '3. low': '58.1300',
  '4. close': '66.1900',
  '5. volume': '32103600'},
 '2000-02-29': {'1. open': '66.2500',
  '2. high': '115.4000',
  '3. low': '66.2500',
  '4. close': '103.9000',
  '5. volume': '23560800'},
 '2000-03-31': {'1. open': '104.5000',
  '2. high': '162.0000',
  '3. low': '90.0000',
  '4. close': '104.0000',
  '5. volume': '41258700'},
 '2000-04-28': {'1. open': '103.0000',
  '2. high': '125.0000',
  '3. low': '73.0600',
  '4. close': '88.6300',
  '5. volume': '32950700'},
 '2000-05-31': {'1. open': '88.6300',
  '2. high': '104.1000',
  '3. low': '54.0000',
  '4. close': '73.6300',
  '5. volume': '77277100'},
 '2000-06-30': {'1. open': '74.4400',
  '2. high': '83.7500',
  '3. low': '62.0000',
  '4. close': '73.7500',
  '5. volume': '128321200'},
 '2000-07-31': {'1. open'

In [ ]:
data, meta_data = ts.get_daily(symbol='A', outputsize='full')

In [ ]:
data = pd.DataFrame(data['4. close'])
data.columns = ['close']
data.head()

In [ ]:
dataset = data.values

In [ ]:
data.shape

In [ ]:
train = dataset[0:4000,:]
valid = dataset[4000:,:]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

In [ ]:
#predicting 246 values, using past 60 from the train data
inputs = data[len(data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

In [ ]:
X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

In [ ]:
inputs.shape[0]

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [ ]:
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms

In [ ]:
#for plotting
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 5]

valid = data.tail(896)
valid['Predictions'] = closing_price
plt.plot(valid[['close','Predictions']])

In [ ]:
valid['close_dif'] = valid['close'].diff()
valid['pred_dif'] = valid['Predictions'].diff()
valid.head()

In [ ]:
valid['type'] = 0
k = valid.loc[(valid['close_dif']>0) & (valid['pred_dif']>0)].index
valid.at[k,'type'] = 1
k = valid.loc[(valid['close_dif']<0) & (valid['pred_dif']<0)].index
valid.at[k,'type'] = 1

In [ ]:
valid.head()

In [ ]:
valid['type'].sum()/valid.shape[0]

In [ ]:
train = data.iloc[0:4000]
valid = data.iloc[4001:]

In [ ]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
driver = webdriver.Chrome()

In [ ]:
url = 'https://www.netfonds.no/quotes/exchange.php?exchange=O'
browser = webdriver.Chrome()

In [ ]:
soup = BeautifulSoup(website_url,'lxml')

In [ ]:
My_table = soup.find('table',{'class':'com'})

In [ ]:
My_table

In [ ]:
ti = My_table.findAll('tr')
ti

In [ ]:
import pandas as pd
# BDay is business day, not birthday...
from pandas.tseries.offsets import BDay

# pd.datetime is an alias for datetime.datetime
today = pd.datetime.today()
lbd = (today - BDay(1)).date()

In [ ]:
df = tickers.loc[tickers['to_date'] == str(lbd)].reset_index(drop=True)
temp = df.head(100)

In [ ]:
yf = YahooFinancials('ABBV')
yf.get_prev_close_price()

In [ ]:
temp.iloc[21]['closing']

In [ ]:
from yahoofinancials import YahooFinancials
from multiprocessing import Pool

t = temp['code']

def f(x):
    yf = YahooFinancials(x)
    return yf.get_prev_close_price()

if __name__ == '__main__':
    with Pool(8) as p:
        closing = p.map(f, t)

temp['closing'] = closing
print('done')

In [ ]:
maxprice = 20
df = temp.loc[temp['closing']<maxprice].reset_index(drop=True)

In [ ]:
df

In [ ]:
import requests
import io
import pandas as pd
import quandl
from datetime import datetime, date,  timedelta

In [ ]:
from yahoofinancials import YahooFinancials
import pandas as pd

In [ ]:
from yahoofinancials import YahooFinancials

ticker = ['AAPL', 'AA']
test = YahooFinancials(ticker)

In [ ]:
d = test.get_historical_price_data('2019-01-01','2019-01-31', 'daily')

In [ ]:
for i in d.keys():
    print(pd.DataFrame(d[i]['prices']))
    break

In [ ]:
d['AA']

In [ ]:
pd.DataFrame.from_dict(d,orient='index')

In [ ]:
def dater(x):
    try:
        return datetime.strptime(x, '%Y-%m-%d').date() 
    except:
        return None

In [ ]:
backup = tickers.copy()

In [ ]:
tickers = backup.copy()

In [ ]:
yesterday = datetime.today().date()-timedelta(1)
tickers['from_date'] = [dater(x) for x in tickers['from_date']]
tickers['to_date'] = [dater(x) for x in tickers['to_date']]
k = [x == yesterday for x in tickers['to_date']]
tickers = tickers.iloc[k].reset_index(drop=True)
tickers.head()

In [ ]:
tickers.shape

In [ ]:
[x[44:53] for x in tickers['description']]

In [ ]:
tickers['description'][0]

In [ ]:
datetime.datetime.strptime(tickers['from_date'][0], '%Y-%m-%d').date()